In [1]:
import numpy as np
from scipy import constants as cst

In [2]:
class general_functions:
    #Define function that normalizes vector
    def normalize(vector):
        #Calculate magnitude of vector
        amp = (np.dot(vector, vector))**0.5
        
        #Normalize
        normalized_vector = vector/amp
        
        return normalized_vector
    
    #Define function that converts from index to x, y, or z coordinate
    #Should work with an array index (x,y,z), but I've only tested with 1 value at a time
    def index2coord(index, space, delta_x):
        #Get the dimensions of space
        dim = space.shape
        
        #Calculate index of the origin
        origin_index = np.divide(np.subtract(dim, 1), 2)
        
        #Calculate distance between given index and origin
        index_spacing = np.subtract(index, origin_index)
    
        #Convert index_spacing into nanometers
        coord = np.multiply(delta_x, index_spacing)

        return coord, index_spacing
    
    #Define function that converts from (x,y,z) coordinates to index
    #Should work with an array index (x,y,z), but I've only tested with 1 value at a time
    def coord2index(coordinates, space, delta_x):
        #Get the dimensions of space
        dim = space.shape
        
        #Convert from nm to index distance
        index_spacing = np.divide(coordinates, delta_x)
        
        #Calculate index of the origin
        origin_index = np.divide(np.subtract(dim, 1), 2)
        
        #Calculate the index
        index = np.add(origin_index, index_spacing)
        
        return index
    
    #Define function that calculates the curl of the E field around a point
    #The code will be more efficient if we don't include any if statements within this function,
    #therefore we will assume everything is given properly and check everything outside of this function
    #Will need to use different function when calculating curl at boundary
    def curl_E(field_space, index, delta_x):
        #Assumes cubic Yee Cells
        #Used for 2D simulation
        
        #Calculate x component
        curl_x = (field_space[index[0], index[1]-1, 2] - field_space[index[0], index[1], 2])/delta_x
        
        #Calculate the y component
        curl_y = (field_space[index[0], index[1], 2] - field_space[index[0]-1, index[1], 2])/delta_x
        
        #Calculate the z component by first calculating the 2 derivatives needed to calculate it
        dEy_dx = (field_space[index[0]-1, index[1], 1] - field_space[index[0], index[1], 1])/delta_x
        dEx_dy = (field_space[index[0], index[1]-1, 0] - field_space[index[0], index[1], 0])/delta_x
        curl_z = dEy_dx - dEx_dy
        
        #The -1 in the indices causes you to reference the next higher x, y, or z value
        
        #Collect all three components into one vector of (x, y, z)
        curl = np.array([curl_x, curl_y, curl_z])
        
        return curl
    
    #Do the same thing as above but for the H field
    def curl_H(field_space, index, delta_x):
        #Calculate x component
        curl_x = (field_space[index[0], index[1], 2] - field_space[index[0], index[1]+1, 2])/delta_x
        
        #Calculate the y component
        curl_y = (field_space[index[0]+1, index[1], 2] - field_space[index[0], index[1], 2])/delta_x
        
        #Calculate the z component by first calculating the 2 derivatives needed to calculate it
        dEy_dx = (field_space[index[0], index[1], 1] - field_space[index[0]+1, index[1], 1])/delta_x
        dEx_dy = (field_space[index[0], index[1], 0] - field_space[index[0], index[1]+1, 0])/delta_x
        curl_z = dEy_dx - dEx_dy
        
        #The +1 in the indices causes you to reference the next lower x, y, or z value
        
        #Collect all three components into one vector of (x, y, z)
        curl = np.array([curl_x, curl_y, curl_z])
        
        return curl
    
#Due to the way we have defined the curl, the collection of H values with the same index as a
#set of E values that in space are shifted in the +x, -y direction